# Tutorial 2: Using SFT and LoRA for Financial Sentiment

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_Financial_Sentiment_CPU.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

We aim to fine-tune an LLM for conducting  **sentiment analysis on financial statements**. The LLM would categorize financial tweets as positive, negative, or neutral. The FinGPT project manages the dataset used in this tutorial. The dataset is a crucial element in this process.

A detailed script for implementation and experimentation is included at the end of this chapter.

In this tutorial, we’ll fine-tune an LLM on a CPU. This is doable with some specific CPUs with optimizations specifically for common operations used in AI, such as Intel’s Xeon CPUs with the use of  [Intel Advanced Matrix Extensions](https://www.intel.com/content/www/us/en/products/docs/accelerator-engines/advanced-matrix-extensions/overview.html)  (AMX). For this tutorial, we create a Compute Engine virtual machine with 64 GB of RAM (as we’ll fine-tune the model on CPU) and the previously mentioned CPUs. Both fine-tuning and inference can be accomplished by leveraging its optimization technologies.

⚠️It’s important to be aware of the costs associated with virtual machines. The total cost will depend on the machine type and the instance’s uptime. Regularly check your costs in the billing section of GCP and spin off your instances when you don’t use them.

💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.